In [54]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")

In [55]:
#entrenamiento = pd.read_csv('https://raw.githubusercontent.com/miguelAfonzalida/TP1-OrgaDatos/master/Entrenamieto_ECI_2020.csv?token=AG4D7WFCVWSLKGHKWKSKGPC7STHPA')
#entrenamiento.head()
df = pd.read_csv('Entrenamieto_ECI_2020.csv')

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

* ##### Contamos en total con 16947 Registros en nuestro set de datos.

* ##### Cuántas Oportunidades distintas tengo?

In [57]:
len(set(df.Opportunity_Name))

9841

 * ##### Veamos primeramente si existen registros duplicadas y para eso comparamos todos los valores existentes cada una de las filas sin tener en cuenta la columna ID que es el id único del registro o fila.

In [58]:
df[df.duplicated(subset=list(df.columns)[1:],keep=False)].shape[0]

619

* ##### Tenemos 619 Registros que se repiten en nuestro set de datos.

* ##### Si nos quedamos con solo con la primera aparición de esos Registros duplicados, cuantos Registros en total debería filtrar?

In [59]:
df[df.duplicated(subset=list(df.columns)[1:],keep='first')].shape[0]

448

* A cuántas oportunidades distintas corresponden esos registros duplicados?

In [60]:
len(set(df[df.duplicated(subset=list(df.columns)[1:],keep=False)].Opportunity_Name))

131

* Vamos a eliminar duplicados excepto los Registros que correspondad a la primera aparición.

In [61]:
df = df.drop_duplicates(subset=list(df.columns)[1:],keep='first') 

In [62]:
len(set(df.ID))

16499

In [63]:
len(set(df.Opportunity_Name))

9841

* ##### Conservamos las mismas cantidades de Oportunidades (9841) pero vemos que al eliminar Registros duplicados, pasamos de 16947 a 16499, osea que eliminamos 448 Registros duplicados que corresponden a un 2,64% de los registros iniciales.

### Limpieza de Regiones y Territorios

In [64]:
df.Region.value_counts()

Japan          4890
EMEA           4649
Americas       3913
APAC           2863
Middle East     184
Name: Region, dtype: int64

In [65]:
# Para los casos en que tenga a Japon como Region y a None como Territory le asigno Japón a Territory, sinó lo dejo como está
df['Territory'] = df.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)

In [66]:
# Dejo Japón solo como territorio y como Región le asigno APAC (Asia-Pacífico)
df.loc[df['Region']=='Japan','Region']='Asia-Pacífico'

In [67]:
# Cambio el nombre de la región Americas por América
df.loc[df['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
df.loc[df['Region']=='EMEA','Region']='Europa'
df.loc[df['Region']=='Middle East','Region']='Oriente Medio'
df.loc[df['Region']=='APAC','Region']='Asia-Pacífico'

In [68]:
df.Region.value_counts()

Asia-Pacífico    7753
Europa           4649
América          3913
Oriente Medio     184
Name: Region, dtype: int64

In [69]:
## Arreglamos la asignación de los territorios a sus correspondientes Regiones
df.loc[df['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
df.loc[df['Territory']=='KSA','Territory']= 'Arabia Saudita'

df.loc[(df['Territory']=='Arabia Saudita')|(df['Territory']=='Jordan')|(df['Territory']=='UAE (Dubai)')\
       |(df['Territory']=='Egypt')|(df['Territory']=='Kuwait'),'Region']= 'Oriente Medio'

df.loc[(df['Territory']=='South America')|(df['Territory']=='SE America')|(df['Territory']=='SW America')\
       |(df['Territory']=='Central America'),'Region']= 'América'

df.loc[df['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

#### Agregamos el Hemisferio según corresponda a cada Territorio.

In [70]:
# Voy a Clasificar los Territorios según los Hemisferios que pertenecen
mis_kw = df['Territory']

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {"Hemisferio Norte": ["NW America","Japan", "India","Italy","Germany","United Kingdom","UAE (Dubai)","Greece"\
                                     ,"Switzerland","Austria","NE America","Belgium","Tunisia",'SE America',"France","Spain","Singapore"\
                                     ,"Philippines","SW America",'Denmark', 'Central America', 'Poland', 'Norway', 'Thailand'\
                                     ,'Portugal', 'South Korea','Netherlands','Kuwait','Arabia Saudita','Cambodia', 'China (PRC)', 'KSA', 'Ireland'\
                                     , 'Jordan','Cyprus', 'Morocco', 'Turkey', 'Sweden', 'Monaco', 'Kazakhstan','Romania', 'Hungary'\
                                     , 'Taiwan','Macedonia', 'Ukraine', 'Czech Republic', 'Belarus','Vietnam', 'Saudi Arabia'\
                                     ,'Bosnia & Herzegovina', 'Kosovo', 'Luxembourg', 'Bulgaria','UAE (Others)', 'Armenia', 'Croatia'\
                                     , 'Malaysia', 'Georgia','Estonia', 'Pakistan', 'Canada', 'Central US', 'Carribean','Malta'\
                                     , 'Albania', 'Oman', 'Burma', 'Slovenia','Egypt']\
               ,"Hemisferio Sur": ["Australia",'South America','Indonesia','South East Asia','Solomon Islands','New Zealand']\
             }
key_depuradas=[]
keyword_processor.add_keywords_from_dict(keyword_dict)
for w in mis_kw:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas.append(w)

In [71]:
len(key_depuradas)

16499

In [72]:
len(df['Territory'])

16499

In [73]:
df['Hemisferio'] = key_depuradas

In [74]:
#Filtro los emisferios para los cuales no tenía un territorio definido
#df = df.loc[(df['Hemisferio']!='None'),:]

In [79]:
df[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
16942,8781,Europa,Austria,Hemisferio Norte
16943,8786,Europa,Austria,Hemisferio Norte
16944,8792,Europa,Austria,Hemisferio Norte
16945,28561,América,NE America,Hemisferio Norte
16946,28318,América,NE America,Hemisferio Norte


#### Agregamos La estación del año en que se crean las oportunidades en funcion del Hemisferio del Terrirorio al que pertenecen.

In [83]:
df['Opportunity_Created_Date']=pd.to_datetime(df['Opportunity_Created_Date'])

In [84]:
def encuentra_Estacion_N(date): 
    year = str(date.year)
    seasons = {'Primavera': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Verano': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Otoño': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Verano']:
        return 'Verano'
    if date in seasons['Otoño']:
        return 'Otoño'
    else:
        return 'Invierno'   
    
def encuentra_Estacion_S(date): 
    year = str(date.year)
    seasons = {'Otoño': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Invierno': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Primavera': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Invierno']:
        return 'Invierno'
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Otoño']:
        return 'Otoño'
    else:
        return 'Verano'     

In [85]:
df.loc[df['Hemisferio']=='Hemisferio Norte','Estación'] = df.loc[df['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)

df.loc[df['Hemisferio']=='Hemisferio Sur','Estación'] = df.loc[df['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

In [86]:
df.Estación.value_counts(normalize=True)

Otoño        0.268300
Primavera    0.265118
Verano       0.233673
Invierno     0.232909
Name: Estación, dtype: float64

In [87]:
df.to_csv("dataTP1.csv", index=False)